## 라이브러리 및 영화 코드 입력

In [15]:
import numpy as np
import pandas as pd
import requests 
from bs4 import BeautifulSoup as bs
from tqdm.notebook import trange
import warnings
warnings.filterwarnings('ignore')

In [54]:
# 영화 코드 리스트 (코드를 리스트안에 직접 넣어야 함)
movie_list = [194204]

## get_review_list 함수 정의

In [55]:
# 영화 코드, 페이지 번호, 댓글 리스트를 매개변수로 설정하면
# 순차적으로 댓글과 여러 정보를 딕셔너리로 저장 후 리스트에 최종으로 담는다. (별다른 리턴값은 없다)
import re

def get_review_list(movie_code, page_no, review_list):
    """
    페이지별로 10개씩 댓글 및 평점을 수집하여 리스트에 넣는다.
    """
    url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false&page={page_no}"    
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    response=requests.get(url,headers=headers) 
    html = bs(response.text,'lxml')
    
    # 크롤링한 데이터에서 댓글(review)과 평점(score) 부분만 불러와서 리스트에 넣는 코드    
    review = html.select('div.score_result > ul > li > div.score_reple > p > span')
    score = html.select('div.score_result > ul > li > div.star_score > em')

    # review에는 관람 여부, 스포일러 여부, 댓글 총 3가지의 데이터가 있다.
    # 상황에 따라 알맞는 데이터를 가져오기 위해 제어변수 num을 설정 및 초기화 하고,
    # 댓글 자체의 순서를 제어할 cnt 변수 초기화한다.
    num = 0
    cnt = 0
    
    # 공감 비공감 전체 리스트
    like = html.select('div.btn_area > a._sympathyButton> strong')
    unlike = html.select('div.btn_area > a._notSympathyButton > strong')  
    
    while (num < len(review)):
        # 정보를 담을 딕셔너리 초기화
        info_dic = {"댓글번호":'', "작성일자":'', "댓글":'', "평점":0, "관람여부":0, "스포여부":0, "공감수":0, "비공감수":0, "id":''} 

        # 댓글번호 추출
        review_no = str(html.select('div.score_reple > dl > dt > em > a')[cnt])
        info_dic["댓글번호"] = int(re.findall('\d+', review_no)[0])
        
        # id 및 작성일자
        info_dic["id"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[0]
        info_dic["작성일자"] = html.select('div.score_reple > dl > dt ')[cnt].text.strip().split('\n\n\n')[1]
        # temp2=html.select('div.score_reple > dl > dt ')[0].text.strip().split('\n\n\n') #여기서 0이 for문에서 바뀔숫자
        
        # 댓글 추출 (관람 여부, 스포일러 여부 추가)
        # 관람 여부 확인 -> 스포일러 여부 확인 -> 댓글 저장 순으로 진행한다.
        # 댓글이 저장되면 최종 저장한 인덱스 그 다음으로 num을 이동시킨다.
        if review[num].text == '관람객':
            info_dic["관람여부"] = 1
            if review[num+1].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
                info_dic["스포여부"] = 1
                info_dic["댓글"] = review[num+2].text.strip()
                num += 3
            else :   
                info_dic["댓글"] = review[num+1].text.strip()
                num += 2
        elif review[num].text == '스포일러가 포함된 감상평입니다. 감상평 보기':
            info_dic["스포여부"] = 1
            info_dic["댓글"] = review[num+1].text.strip()
            num += 2
        else :
            info_dic["댓글"] = review[num].text.strip()
            num += 1

        # 평점 추출
        info_dic["평점"] = score[cnt].text.strip()
        
        # 공감, 비공감수 추출
        info_dic["공감수"] = int(re.findall('\d+',str(like[cnt]))[1])
        info_dic["비공감수"] = int(re.findall('\d+',str(unlike[cnt]))[1])            
        
        # 댓글의 모든 정보를 넣었으니 순서를 다음으로 넘긴다.
        cnt += 1
        # 정보가 담긴 딕셔너리를 리스트에 저장
        review_list.append(info_dic)


## get_page_num 함수 정의

In [57]:
# 영화 코드를 매개변수로 설정하면
# 해당 영화의 전체 페이지 수 리턴 받는 함수
def get_comment_num(movie_code):
    url=f"https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code={movie_code}&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false"    
    headers={'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'}
    response=requests.get(url,headers=headers) 
    html = bs(response.text,'lxml')

    # 크롤링한 데이터에서 전체 댓글 수를 나타낸 부분을 가져온다
    total = html.select('div.score_total > strong.total > em')
    total_cnt = int(total[0].text.replace(',',''))
    
    # 각 페이지마다 댓글이 10개씩 있으므로
    # 댓글수(cnt)가 10의 배수이면 전체 페이지는 나누기 10의 값이고
    # 10의 배수가 아니면 나누기 10을 한 값에 1페이지를 더해준다.
    page_end = int(total_cnt / 10)
    if total_cnt % 10 != 0:
        page_end += 1
    
    return page_end

## 영화 코드 반복을 통한 전체 영화 정보 저장

In [58]:
review_list = []
for i in trange(len(movie_list)):
    # 전체 페이지 수 재설정
    page_end = get_comment_num(movie_list[i])
    
    # 처음부터 마지막 페이지까지 반복
    for j in trange(page_end):
        # get_review_list 함수를 통해 댓글 정보들을 리스트에 담는다
        get_review_list(movie_list[i], j+1, review_list)


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/181 [00:00<?, ?it/s]

## 리뷰와 평점 병합해주기 

In [59]:
review_df=pd.DataFrame(review_list)
review_df

,댓글번호,작성일자,댓글,평점,관람여부,스포여부,공감수,비공감수,id
0,17927129,2022.01.26 23:41,"네이버 평점 두번째 남겨보고, 여기저기 평점 쓰면서 1점 처음 남겨봅니다. 연출, ...",1,1,0,319,165,Neth(yswa****)
1,17926196,2022.01.26 17:46,별로에요...중간중간 대사가 잘 안들리기도 하고 좀 유치...,3,1,0,247,116,vivi(vivi****)
2,17926313,2022.01.26 18:31,그냥 1편으로 남겼어도 되지 않을까 싶습니다. 영상미는 좋았는데 스토리가 너무 지지...,4,1,0,242,135,zin(gywl****)
3,17926961,2022.01.26 22:51,정말..너무 별로였어요 시간아깝고..유치하고...,2,1,0,207,106,lIIllIlI(e962****)
4,17925703,2022.01.26 13:34,재미있네요. 장르는 로맨스 코미디 모험 같네요,10,1,0,212,124,shi(shin****)
...,...,...,...,...,...,...,...,...,...
1804,17925258,2022.01.26 09:47,해적1 생각하고 왔다가 씁쓸해지네요 손예진 연기보다가 한효주 연기보니 비교가 많이 ...,2,0,0,2,12,dkwnaTlekd(love****)
1805,17925271,2022.01.26 09:55,진심으로 말합니다 29년 인생중 최악이고 이광수님 런닝맨 뛰세요쥔나게 뛰세요 그냥 ...,1,0,0,3,14,hsh7****
1806,17925265,2022.01.26 09:50,영상미 괜챦아요 그러나 한효주씨 연기 너무 어색하고 눈빛도 이상하고 스토리도 해적1...,2,0,0,2,14,sjpa****
1807,17926674,2022.01.26 21:15,한효주 연기도 괜찮았고 가족들이 보기 좋은 영화였음! 설에 가족들이랑 다같이 보면 ...,8,1,1,13,29,grapefruit(dkfm****)


In [78]:
review_df.to_csv("댓글1.csv")

## 추가작업
크롤링 한 데이터 외에도 '영화이름', '영화코드', '개봉일' 데이터를 추가로 넣고,  
공감과 비공감의 차이를 담을 '공감차이' 컬럼을 만들기 위한 추가 작업

In [79]:
# 모든 크롤링 데이터를 불러서 concat으로 붙인다.
df1 = pd.read_csv('./data/댓글1.csv')
df2 = pd.read_csv('./data/댓글2.csv')
df3 = pd.read_csv('./data/댓글3.csv')
df4 = pd.read_csv('./data/댓글4.csv')
df5 = pd.read_csv('./data/댓글5.csv')
df6 = pd.read_csv('./data/댓글6.csv')
df7 = pd.read_csv('./data/댓글7.csv')

total_df = pd.concat([df1, df2, df3, df4 ,df5 ,df6 ,df7])

In [81]:
# 인덱스를 처음부터 정리하면 파일을 만들었어야 했는데...
# 그러지 못했으니 지금에서라도 인덱스들을 잘 정리하자
total_df.drop('index', axis=1, inplace=True)
total_df.reset_index(drop=True, inplace=True)
total_df

In [82]:
# 영화이름, 영화코드, 개봉일을 인덱스 순서에 맞게 리스트에 넣는다.
movie_list = ['타이거마스크','쇼미더고스트','습도다소높음','귀신','미션파서블','완벽한타인'
              ,'럭키','미녀는괴로워','오케이마담','굿바이싱글','미쓰와이프','걸캅스','수상한그녀'
              ,'정직한후보','과속스캔들','마파도','댄싱퀸','써니','피끓는청춘','퍼펙트맨'
              ,'탐정:리턴즈','탐정:더비기닝','보안관','구타유발자들','게이트','미드나잇인파리'
              ,'예스맨','원더풀고스트','위험한상견례','형','라디오스타','이장과군수','헬머니'
              ,'가문의영광5','상사부일체','내안의그놈','승리호','인랑','호빗','가려진시간'
              ,'당신,거기있어줄래요','디워','트랜스포머','고질라:킹오브몬스터','배트맨대슈퍼맨'
              ,'매트릭스','닥터두리틀','그린델왈드의범죄','미스터고','이상한나라의엘리스'
              ,'판타스틱4','설국열차','나는전설이다','초능력자','염력','루시드드림'
              ,'인류멸망보고서','7광구','조작된도시','더폰','흡혈괴마의비밀','사라진놉의딸'
              ,'죽지않는인간들의밤','창궐','화이','지아이조','숨바꼭질','매드맥스:분노의도로'
              ,'레전드']
movie_code = ['210895','206220','204175','194887','189124','167638','140695','39157'
              ,'182234','139700','129406','174065','107924','186821','51143','39422'
              ,'83268','76016','105521','180390','159892','124201','146524','56469'
              ,'172005','74610','45944','152156','77125','142803','58088','61639'
              ,'123470','96969','65058','164172','186342','109906','97816','141259'
              ,'145292','39569','123630','137327','109960','191600','159866','154255'
              ,'75397','50163','118968','62328','64191','73344','153651'
              ,'130713','61004','48246','127382','134982','165748','123596','183854'
              ,'160487','98467','72405','102824','77768','129012']
openday_list = ['2021.12.30','2021.09.09','2021.09.01','2021.08.25','2021.02.17'
                ,'2018.10.31','2016.10.13','2006.12.14','2020.08.12','2016.06.29'
                ,'2015.08.13','2019.05.09','2014.01.22','2020.02.12','2008.12.03'
                ,'2005.03.10','2012.01.18','2011.05.04','2014.01.22','2019.10.02'
                ,'2018.06.13','2015.09.24','2017.05.03','2006.05.31','2018.02.28'
                ,'2012.07.05','2008.12.17','2018.09.26','2011.03.31','2016.11.23'
                ,'2006.09.28','2007.03.29','2015.03.05','2012.12.19','2007.09.19'
                ,'2019.01.09','2021.02.05','2018.07.25','2014.12.17','2016.11.16'
                ,'2016.12.14','2007.08.01','2017.06.21','2019.05.29','2016.03.24'
                ,'2021.12.22','2020.01.08','2018.11.14','2013.07.17','2010.03.04'
                ,'2015.08.20','2013.08.01','2007.12.12','2010.11.10','2018.01.31'
                ,'2017.02.22','2012.04.11','2011.08.04','2017.02.09','2015.10.22'
                ,'2018.02.08','2015.02.11','2020.09.29','2018.10.25','2013.10.09'
                ,'2013.03.28','2013.08.14','2015.05.14','2015.12.10']


In [ ]:
# 데이터프레임에 3가지 컬럼을 생성하고, 첫번째 행에는 미리 영화 정보들을 넣는다.
# 그리고 공감수-비공감수의 값을 '공감차이' 컬럼을 생성하여 넣는다.
cnt = 0
total_df['영화이름'] = ''
total_df['영화이름'][0] = movie_list[cnt]
total_df['영화코드'] = 0
total_df['영화코드'][0] = movie_code[cnt]
total_df['개봉일'] = ''
total_df['개봉일'][0] = openday_list[cnt]
total_df['공감차이'] = total_df['공감수'] - total_df['비공감수']

In [68]:
# for문으로 영화이름, 코드, 개봉일을 넣는 구문
# 최초 크롤링 데이터가 공감차이 순으로 정렬되어 있으므로 이전 행보다 다음행이 공감차이 숫자가 작다.
# 그런데 이전 행보다 다음 행의 공감차이 숫자가 크면 다음 영화의 시작이므로 cnt를 올려서
# 리스트에서 다음 영화를 인덱싱하게 조정하여 for문을 돌린다.
#cnt = 48
for i in trange(459087,len(total_df)):
    if (total_df['공감차이'][i] - total_df['공감차이'][i-1]) <= 0:
        total_df['영화이름'][i] = movie_list[cnt]
        total_df['영화코드'][i] = movie_code[cnt]
        total_df['개봉일'][i] = openday_list[cnt]
    else:
        cnt += 1
        total_df['영화이름'][i] = movie_list[cnt]
        total_df['영화코드'][i] = movie_code[cnt]
        total_df['개봉일'][i] = openday_list[cnt]

  0%|          | 0/202095 [00:00<?, ?it/s]

In [182]:
total_df = total_df[['댓글번호', '작성일자', '영화코드', '영화이름','댓글', '평점'
                     , '관람여부', '스포여부', '공감수', '비공감수', '공감차이', 'id']]
total_df

,댓글번호,작성일자,영화코드,영화이름,댓글,평점,관람여부,스포여부,공감수,비공감수,공감차이,id
0,17882776,2021.12.30 19:49,210895,타이거마스크,ㅋㅋ 돈있으면 나줘라그냥..이거 반칙왕 복제품이냐? ㅋㅋ 아놔.. 평점댓글알바 오지고요!,1,0,0,30,13,17,Cffvvhddcvbi(jjho****)
1,17882946,2021.12.30 21:28,210895,타이거마스크,누적관객수 2명.너무해,5,0,0,21,7,14,배짱(oari****)
2,17882792,2021.12.30 20:00,210895,타이거마스크,"스토리는 발로썼고, 대사는 오그라들어서 시공간이 없어질꺼같고, 연기는 초등학생 데려...",2,0,0,26,14,12,hdt7****
3,17884880,2021.12.31 22:36,210895,타이거마스크,"한국의 B급영화를 생각하고 봤습니다.한국의 영화리그 중 A,B 급이 아닌AA,BB급...",2,0,0,21,10,11,마지탐방(hake****)
4,17883636,2021.12.31 03:52,210895,타이거마스크,와.. 별점을 줄수가 없는 영화다... 비급무비라고 생각하고 봐도 이건 아닌데 별점...,1,0,0,23,12,11,핸디하로 커스터머(inno****)
...,...,...,...,...,...,...,...,...,...,...,...,...
661920,17925263,2022.01.26 09:49,194204,해적,채수빈만 알아가는 영화..아침부터 봤는데 시간 아까움...,2,0,0,1,8,-7,630k****
661921,17925579,2022.01.26 12:40,194204,해적,이광수가 다한영화!! 한효주 좀 어색한것 같아요ㅠㅠ,6,1,0,19,28,-9,ARUM KIM(dkfm****)
661922,17925258,2022.01.26 09:47,194204,해적,해적1 생각하고 왔다가 씁쓸해지네요 손예진 연기보다가 한효주 연기보니 비교가 많이 ...,2,0,0,2,12,-10,dkwnaTlekd(love****)
661923,17925271,2022.01.26 09:55,194204,해적,진심으로 말합니다 29년 인생중 최악이고 이광수님 런닝맨 뛰세요쥔나게 뛰세요 그냥 ...,1,0,0,2,14,-12,hsh7****


In [ ]:
review_df.to_csv("review_jaein.csv")